In [1]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-community==0.3.25 \
  langchain-openai==0.3.22 \
  numexpr==2.11.0

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/learn/generation/langchain/handbook)

# Getting Started with Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

Nowadays, chains are mostly built using **L**ang**C**hain **E**xpression **L**anguage (LCEL) rather that using objects like the `LLMChain`. So, when we refer to the `LLMChain` we are references this _past_ object but nowadays this would be implemented via LCEL syntax - which we'll explore soon.

Let's start by importing all the libraries that we'll be using in this example.

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use throughout the notebook, just input your [OpenAI API key](https://platform.openai.com/api-keys) below.

In [2]:
from IPython.display import display, Markdown
import os
from getpass import getpass

# must enter API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "langchain-pinecone-io-walkthrough-chains"

In [3]:
from langchain_openai import ChatOpenAI

# initialize the models
llm = ChatOpenAI(
    model_name="gpt-5-mini",
    temperature=1.0
)

## Chains and LCEL

Chains in LangChain are now built using the LangChain Expression Language (LCEL), which takes a declarative approach to combining components. Instead of using predefined chain classes, LCEL lets you compose chains using the `|` operator and other composition primitives.

### Types of Chain Composition

1. **Sequential Chains** (`|` operator)
   - Chain components one after another
   - Example: `prompt | llm | output_parser`

2. **Parallel Chains** (`RunnableParallel`)
   - Run multiple operations concurrently
   - Example: Running multiple prompts or retrievers in parallel

3. **Complex Workflows**
   - For more complex scenarios involving branching, cycles, or multiple agents
   - Recommended to use LangGraph instead of LCEL directly

Let's start with a simple example: creating a sequential math chain that can handle calculations...

In [4]:
import numexpr
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# Create a function to handle calculations
def calculate(expression: str) -> str:
    """Calculate using numexpr, with support for basic math operations."""
    try:
        result = float(numexpr.evaluate(expression))
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

# Wrap our calculation function with RunnableLambda for explicit LCEL pattern
calculate_runnable = RunnableLambda(calculate)

# Create the chain using LCEL with explicit RunnableLambda
math_chain = (
    prompt
    | llm # ChatOpenAI(temperature=0)
    | StrOutputParser()  # Convert to string
    | calculate_runnable  # Our calculation function wrapped in RunnableLambda
)

# Use the chain with our example
response = math_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)

The result is: 2.4116004626599237


Let's see what is going on here. The chain processes our input through several sequential steps:

1. The prompt template formats our question
2. The LLM converts it to a mathematical expression
3. The StrOutputParser ensures we get a clean string
4. Finally, our calculate function computes the result

But how did the LLM know to return just the mathematical expression? 🤔

**Enter prompts**

The question we send isn't the only input the LLM receives 😉. Look at our prompt template:


In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful math assistant. When given a math problem, respond ONLY with the mathematical expression that would solve it. For example, if asked 'What is 2 raised to the 3rd power?', respond only with '2**3'."),
    ("user", "{question}")
])

The system message explicitly instructs the LLM to return only the mathematical expression. Without this context, the LLM would try to calculate the result itself. Let's test this by trying without the system message:


In [6]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("user", "{question}")
])

basic_chain = (
    prompt
    | llm # ChatOpenAI(temperature=0)
    | StrOutputParser()
)

response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)


13^0.3432 ≈ 2.41160095 (≈ 2.4116 to 4 decimal places).


It seems that gpt-5-mini is has the architecture to use tools and think step by step so that it find the right answer. Let's simulate a lighter llm by adding a system prompt message which tells the llm to not use any external tools.

In [7]:
# Simple prompt without guidance
prompt = ChatPromptTemplate.from_messages([
    ("system", "Do not use any external tools, api calls, or plugins. Just answer the question directly."),
    ("user", "{question}")
])

# same basic chain as before
response = basic_chain.invoke({
    "question": "What is 13 raised to the .3432 power?"
})
print(response)  # The LLM tries to calculate it directly and gets it wrong!

13^0.3432 = e^{0.3432 ln 13} ≈ e^{0.8802906195} ≈ 2.41156

(Approximately 2.41156; let me know if you want more digits.)


Let's ignore that the model is inherently good enough to ansewr this question supposedly without help. gpt-5-mini must be robust in other ways that are beyond the scope of prompt engineering. We would see the difference clearly if we used a smaller model. Anyway, this demonstrates the power of prompting in LCEL: by carefully designing our prompts, we can guide the LLM's behavior precisely.

The beauty of LCEL's sequential composition is how clearly we can see each step in the chain:


In [8]:
math_chain = (
    prompt                         # Step 1: Format the input with our system message
    | llm                          # Step 2: Get mathematical expression from LLM
    | StrOutputParser()            # Step 3: Convert to clean string
    | calculate                    # Step 4: Evaluate the expression
)

Each step flows naturally into the next using the `|` operator, making it easy to understand and modify the chain's behavior. This is much more flexible than the old approach of using predefined chain classes - we can easily add, remove, or modify steps as needed!

*_Note: The `calculate` function uses `numexpr` to safely evaluate mathematical expressions without needing a full Python REPL (Read-Eval-Print Loop)._

### Building Complex Chains with LCEL

Let's build a more complex example that shows how to combine different components using LCEL. We'll create a chain that cleans up messy text and then paraphrases it in a specific style.

First, let's create a function to clean up text by removing extra spaces and newlines. In LCEL, we can use regular functions directly in our chain:

In [9]:
import re

def clean_text(text: str) -> str:
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text

Now, let's create our prompt template for the paraphrasing:

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative writing assistant."),
    ("user", """Please paraphrase this text in the style of {style}:

{text}""")
])

Now we can combine everything into a sequential chain using LCEL's `|` operator. The beauty of LCEL is how naturally we can compose these components:


In [11]:
# Create the chain using LCEL
style_chain = (
    {
        "text": lambda x: clean_text(x["text"]),  # Extract and clean the text from input dict
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | prompt  # Format with our template
    | llm  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

In [12]:
# Alternative using Runnable Passthrough
from langchain_core.runnables import RunnablePassthrough
style_chain = (
    {
        "text": RunnablePassthrough(),  # Pass the entire input dictionary
        "style": lambda x: x["style"]  # Extract style from input dict
    }
    | {"text": lambda x: clean_text(x["text"]), "style": RunnablePassthrough()}  # Apply clean_text and pass through style
    | prompt  # Format with our template
    | llm  # Generate creative paraphrase
    | StrOutputParser()  # Convert to string
)

# Our input text with messy spacing
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

# Run the chain
response = style_chain.invoke({
    "text": input_text,
    "style": "a 90s rapper"
})
print(response)

Yo — chains be the crew that link the pieces tight,
Stackin' components so the app runs right.
You feed in the user vibe, we lace it with a PromptTemplate,
Polish the words, hand 'em off to the LLM gate.
Want somethin' dope and complex? We fuse chains like tracks,
Or blend 'em with other modules, put the system on wax.
Keep it modular, keep the flow — one smooth application, no slack.


Let's look at how this chain works:

1. The dictionary `{"text": clean_text, "style": lambda x: x}` processes our inputs in parallel using `RunnableParallel`
2. The `|` operator connects each component, showing the clear flow of data
3. Each step in the chain serves a specific purpose and is easily modifiable
4. The components work together seamlessly to process and transform the text

This demonstrates how LCEL lets us compose simple components into powerful chains while keeping the code readable and maintainable. Whether you're processing text, generating content, or building complex workflows, LCEL's composition primitives make it easy to build exactly what you need! 🔥

### Using RunnableParallel and RunnablePassthrough

Let's explore how to use `RunnableParallel` for running multiple operations concurrently and `RunnablePassthrough` for passing data through unchanged:

In [13]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Create two different analysis prompts
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert. Analyze the emotional tone."),
    ("user", "What's the sentiment of: {text}")
])

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a summarization expert."),
    ("user", "Summarize in one sentence: {text}")
])

# Use RunnableParallel to run both analyses simultaneously
analysis_chain = RunnableParallel(
    {
        "sentiment": sentiment_prompt | llm | StrOutputParser(),
        "summary": summary_prompt | llm | StrOutputParser(),
        "original": RunnablePassthrough()  # Pass through the original input
    }
)

# Test it
sample_text = {"text": "The product exceeded my expectations. Great quality!"}
results = analysis_chain.invoke(sample_text)

print("Sentiment:", results["sentiment"])
print("Summary:", results["summary"])
print("Original:", results["original"]["text"])

Sentiment: Sentiment: Positive — very positive (high confidence).

Why: Phrases like "exceeded my expectations" and "Great quality!" express strong praise and satisfaction. Target/aspect: product quality. Emotion: approval/delight. Subjectivity: clearly subjective opinion. Suggested score: ~0.9/1.0.
Summary: The product exceeded expectations and is of great quality.
Original: The product exceeded my expectations. Great quality!


### Batch Processing with LCEL

LCEL chains support efficient batch processing using the `.batch()` method:

In [14]:
# Create a simple question-answering chain
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer concisely."),
    ("user", "{question}")
])

qa_chain = qa_prompt | llm | StrOutputParser()

# Batch of questions
questions = [
    {"question": "What is the capital of France?"},
    {"question": "Who wrote Romeo and Juliet?"},
    {"question": "What is the speed of light?"}
]

# Process all questions in batch
answers = qa_chain.batch(questions)

# Display results
for q, a in zip(questions, answers):
    print(f"Q: {q['question']}")
    print(f"A: {a}\n")

Q: What is the capital of France?
A: The capital of France is Paris.

Q: Who wrote Romeo and Juliet?
A: Romeo and Juliet was written by William Shakespeare (usually dated to the mid-1590s).

Q: What is the speed of light?
A: The speed of light in vacuum is exactly 299,792,458 m/s (by definition).  
Approximate values: 3.00×10^8 m/s, 299,792.458 km/s, ≈186,282.4 miles/s (≈670.6 million mph).  

(Light travels slower in materials; c is the invariant speed limit in relativity.)



That's it for this example on chains with LCEL.

---

Key concepts: Chaining with LCEL using ChatPromptTemplate, StrOutputParser, RunnableLambda, RunnableParallel, RunnablePassthrough, and Batch processing.